In [1]:
import gzip
import numpy as np
from functools import lru_cache
from pprint import pprint
from dahuffman import HuffmanCodec
import pandas as pd
import random
from prettytable import PrettyTable
import lz4.frame
from typing import Tuple, Union, List

In [2]:
@lru_cache(maxsize=None)
def gzip_compress(x):
    return gzip.compress(x.encode())

In [3]:
def huffman_code(x):
    codec = HuffmanCodec.from_data(x)
    code = codec.encode(x)
    return code

In [4]:
def lz4_encode(x:str):
    return lz4.frame.compress(x.encode())

In [5]:
def bwt(s: str) -> str:
    """Apply Burrows–Wheeler transform to input string."""
    assert "\002" not in s and "\003" not in s, "Input string cannot contain STX and ETX characters"
    s = "\002" + s + "\003"  # Add start and end of text marker
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of string
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string

# mtfwiki.py
# Instead of always transmitting an "original" dictionary, it is simpler to just agree on an initial set.
# Here we use the 256 possible values of a byte:
common_dictionary = list(range(256))

def mtf_encode(plain_text: str) -> List[int]:
    # Change to bytes for 256.
    plain_text = plain_text.encode("utf-8")

    # Changing the common dictionary is a bad idea. Make a copy.
    dictionary = common_dictionary.copy()

    # Transformation
    compressed_text = list()          # Regular array
    rank = 0

    # Read in each character
    for c in plain_text:
        rank = dictionary.index(c)    # Find the rank of the character in the dictionary [O(k)]
        compressed_text.append(rank)  # Update the encoded text

        # Update the dictionary [O(k)]
        dictionary.pop(rank)
        dictionary.insert(0, c)

    return compressed_text            # Return the encoded text

def bzip(text: str) -> List[int]:
    return mtf_encode(bwt(text))

In [6]:
compress_functions = {"gzip_compress": gzip_compress,
                      "lz4_encode": lz4_encode,
                      "huffman_code": huffman_code,
                      "bzip": bzip}

In [7]:
def ncd(x1, x2, function_name):
    compression_function = compress_functions[function_name]
    if x1 == x2:
        return 1e5
    cx1 = len(compression_function(x1))
    cx2 = len(compression_function(x2))
    x1x2 = x1 + x2
    cx1x2 = len(compression_function(x1x2))
    dist = (cx1x2 - min(cx1, cx2)) / max(cx1, cx2)
    return dist

In [8]:
classes = {
    "Sports": [
"Football is a popular sport enjoyed by millions worldwide. The players engage in fierce competition with the aim of scoring the most goals. The tactics and strategies employed by different teams often play a critical role in their success. The exhilarating experience of watching a football match in a stadium full of passionate fans is hard to match.",
"Cricket is a sport that requires a blend of physical fitness and strategic planning. It is loved by millions around the globe, especially in countries like England, Australia and India. The thrilling run chases, astounding catches, and nail-biting finishes are what make cricket a spectacular sport."
    ],
    "Astronomy": [
"The vastness of the universe is explored through astronomy. The study encompasses understanding celestial bodies such as the sun, stars, planets, and galaxies, and extraterrestrial phenomena. The quest for knowledge about the universe has led to iconic discoveries and advancements in technology.",
"Astrophysics, a branch of astronomy, seeks to understand the workings of stars, galaxies, and the cosmos as a whole. There are countless mysteries yet to be unraveled about the universe, from dark energy to the possibility of life elsewhere in the cosmos."],
    "Cooking": [
"Cooking is an art that brings a fusion of flavors, textures, and aroma to the table. Several cooking methods, such as grilling, roasting, baking, boiling, and frying are used to create a culinary masterpiece. The mastery of cooking involves a fine understanding of ingredients and their intrinsic properties.",
"Gastronomy focuses on the discipline of cooking, focusing on the artistic, technical, and cultural aspects of food. Mastery in this field requires an appreciation for quality ingredients, technique, and cultural tradition. Experimentation adds to the artistry of gastronomy, resulting in unique, delectable dishes."    ],
"Gardening": [
        "In her beautiful garden, Mary cared for various flowering plants and shrubs diligently. She knew the importance of regular watering, appropriate sunlight, and quality fertilizers. She nurtured roses, tulips, and sunflowers with utmost love and care.",

"Sarah was a passionate gardener, tending to an array of blooms and bushes with attentiveness. She acknowledged the significance of consistent irrigation, needed sunlight exposure, and effective fertilizers. She fostered roses, tulips, and sunflowers with abundant affection and care."
    ]
    
}


In [9]:
def score():
    compress_methods = {"gzip_compress": [], "huffman_code": [], "lz4_encode": [], "bzip": []}
    
    for class_name, samples in classes.items():
        for i, sample in enumerate(samples):
            print(f'Processing sample {i+1} from {class_name}.')

            table = PrettyTable()
            table.field_names = ["Sample", "Class", "Distance"]

            for compress_method, results in compress_methods.items():
                min_distance = float('inf')
                min_class = None
                min_sample_index = None

                for other_class_name, other_samples in classes.items():
                    for j, other_sample in enumerate(other_samples):
                        if other_class_name == class_name and i == j:
                            continue
                        distance = ncd(sample, other_sample, compress_method)
                        table.add_row([j+1, other_class_name, distance])
                        if distance < min_distance:
                            min_distance = distance
                            min_class = other_class_name
                            min_sample_index = j

                print(table)
                print(f'Closest sample is {min_sample_index+1} from {min_class} with distance {min_distance}.')
                compress_methods[compress_method].append({"Class": min_class, "Distance": min_distance})
                print('\n')

    df = pd.DataFrame(compress_methods)
    return df


df = score()


Processing sample 1 from Sports.
+--------+-----------+--------------------+
| Sample |   Class   |      Distance      |
+--------+-----------+--------------------+
|   2    |   Sports  | 0.7370689655172413 |
|   1    | Astronomy | 0.7974137931034483 |
|   2    | Astronomy | 0.7974137931034483 |
|   1    |  Cooking  | 0.7801724137931034 |
|   2    |  Cooking  | 0.7801724137931034 |
|   1    | Gardening | 0.7758620689655172 |
|   2    | Gardening | 0.7887931034482759 |
+--------+-----------+--------------------+
Closest sample is 2 from Sports with distance 0.7370689655172413.


+--------+-----------+--------------------+
| Sample |   Class   |      Distance      |
+--------+-----------+--------------------+
|   2    |   Sports  | 0.7370689655172413 |
|   1    | Astronomy | 0.7974137931034483 |
|   2    | Astronomy | 0.7974137931034483 |
|   1    |  Cooking  | 0.7801724137931034 |
|   2    |  Cooking  | 0.7801724137931034 |
|   1    | Gardening | 0.7758620689655172 |
|   2    | Gardenin

In [10]:
df

,gzip_compress,huffman_code,lz4_encode,bzip
0,"{'Class': 'Sports', 'Distance': 0.737068965517...","{'Class': 'Astronomy', 'Distance': 1.016129032...","{'Class': 'Sports', 'Distance': 0.802259887005...","{'Class': 'Sports', 'Distance': 0.994334277620..."
1,"{'Class': 'Sports', 'Distance': 0.732758620689...","{'Class': 'Cooking', 'Distance': 1.00606060606...","{'Class': 'Sports', 'Distance': 0.779661016949...","{'Class': 'Astronomy', 'Distance': 0.993355481..."
2,"{'Class': 'Astronomy', 'Distance': 0.671641791...","{'Class': 'Cooking', 'Distance': 1.00606060606...","{'Class': 'Astronomy', 'Distance': 0.745874587...","{'Class': 'Astronomy', 'Distance': 0.993288590..."
3,"{'Class': 'Astronomy', 'Distance': 0.666666666...","{'Class': 'Cooking', 'Distance': 1.00606060606...","{'Class': 'Astronomy', 'Distance': 0.729372937...","{'Class': 'Gardening', 'Distance': 0.992217898..."
4,"{'Class': 'Cooking', 'Distance': 0.70673076923...","{'Class': 'Sports', 'Distance': 1.006060606060...","{'Class': 'Astronomy', 'Distance': 0.809061488...","{'Class': 'Sports', 'Distance': 0.993548387096..."
5,"{'Class': 'Cooking', 'Distance': 0.70673076923...","{'Class': 'Sports', 'Distance': 1.011904761904...","{'Class': 'Cooking', 'Distance': 0.78964401294...","{'Class': 'Sports', 'Distance': 0.993670886075..."
6,"{'Class': 'Gardening', 'Distance': 0.604060913...","{'Class': 'Cooking', 'Distance': 1.00606060606...","{'Class': 'Gardening', 'Distance': 0.680412371...","{'Class': 'Astronomy', 'Distance': 0.992217898..."
7,"{'Class': 'Gardening', 'Distance': 0.593908629...","{'Class': 'Cooking', 'Distance': 1.00606060606...","{'Class': 'Gardening', 'Distance': 0.676975945...","{'Class': 'Astronomy', 'Distance': 0.992982456..."
